# CREATE KAGGLE SUBMISSION NOTEBOOK


### FOR GOOGLE COLLAB USERS ONLY: Mount Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "drive/My Drive/JPNotebook/cs/"

In [ ]:
import pandas as pd
import numpy  as np
import requests
import math
import json
from pathlib import Path
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing  import OneHotEncoder
from sklearn.compose        import ColumnTransformer
from sklearn.pipeline       import Pipeline
from sklearn.metrics        import mean_absolute_error, mean_squared_error, r2_score

from xgboost import XGBRegressor
import joblib

# Import Data


In [ ]:
DATA_PATH   = Path(path + "northcarolina.csv")
MODEL_PATH  = Path(path + "northCarolina.joblib")

TARGET_COL  = "Energy (kWh)"
DATETIME_COL = "StartDateTime"
NAME_COL      = "Station Name"
ID_COL      = "StationID"
NUMERIC_COLS = ["Latitude", "Longitude", "Duration_min" ]

RANDOM_SEED = 17

# Build Model


In [ ]:
from zoneinfo import ZoneInfo   # Python 3.9+

# chat gpt gnerated function:
def add_time_features(df: pd.DataFrame, time_col: str) -> pd.DataFrame:
    ts_utc   = pd.to_datetime(df[time_col], errors="coerce", utc=True)
    ts_local = ts_utc.dt.tz_convert(ZoneInfo("America/New_York"))

    df["hour"]       = ts_local.dt.hour
    df["dow"]        = ts_local.dt.dayofweek
    df["month"]      = ts_local.dt.month
    df["is_weekend"] = (df["dow"] >= 5).astype(int)

    # Cyclical encodings
    df["hour_sin"]  = np.sin(2 * np.pi * df["hour"]  / 24)
    df["hour_cos"]  = np.cos(2 * np.pi * df["hour"]  / 24)
    df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
    df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

    df["Duration_min"] = (
    pd.to_timedelta(df["Charging Time"])
      .dt.total_seconds()
      / 60
    )

    return df


In [ ]:
df = pd.read_csv(DATA_PATH)
print(f"Loaded {len(df):,} sessions.")

df["StationID"] = df[NAME_COL].astype("category").cat.codes

Loaded 20,142 sessions.


In [ ]:
def fetch_api(lat: float, lon: float, dist: float, max_results: int = 500, api_key: str = None):
    if api_key is None:
        raise ValueError("API key must be provided")

    url = (
        "https://api.openchargemap.io/v3/poi/?output=json"
        f"&latitude={lat}&longitude={lon}"
        f"&distance={dist}&maxresults={max_results}&key={api_key}"
    )
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    return data
def getpoints(data):
    points = []
    for item in data:
        info = item.get("AddressInfo", {})
        plat = info.get("Latitude", 0.0)
        plon = info.get("Longitude", 0.0)
        points.append((plat, plon))
    return points

In [ ]:
API_KEY = "39dc9e88-98fb-449f-ae0f-f44d99a4fc5b"
poi_points = fetch_api(35.7893, -78.8022, 20, max_results=500, api_key=API_KEY)


In [ ]:
def api_to_df(api_data: list[dict]) -> pd.DataFrame:
    records = []
    for item in api_data:
        ai = item["AddressInfo"]
        conns = item.get("Connections", [])
        if conns:
            power = conns[0].get("PowerKW", None)
            amps = conns[0].get("Amps", None)
            voltage = conns[0].get("Voltage", None)
            currentType = conns[0].get("CurrentTypeID", None)
            conn_type = conns[0].get("ConnectionTypeID", None)
            qty = conns[0].get("Quantity", 1)
        else:
            currentType = voltage = amps = power = conn_type = qty = None

        operator_info = item.get("OperatorInfo") or {}
        records.append({
            "StationAPI_ID": ai["ID"],
            "Latitude":      ai["Latitude"],
            "Longitude":     ai["Longitude"],
            "Power_kW":      power,
            "Amps":      amps,
            "Volate": voltage,
            "NumConnectors": qty,
            "CurrentTypeID": currentType,
            "ConnectionTypeID": conn_type,
            "UsageTypeID":   item.get("UsageTypeID", None),
            "Model Number":  operator_info.get("Title", None),
        })
    return pd.DataFrame(records)

api_df = api_to_df(poi_points)

In [ ]:
points = getpoints(poi_points)
print(points)

[(35.7968864, -78.8080444), (35.79866661495119, -78.7992852165608), (35.797205, -78.813656), (35.79642823011835, -78.8164944400895), (35.773958, -78.803281), (35.801074720113874, -78.81448849833393), (35.790599, -78.780497), (35.784842, -78.780842), (35.782687, -78.781277), (35.785228, -78.779495), (35.789215, -78.77882), (35.783205, -78.779366), (35.781304, -78.825552), (35.788277, -78.773542), (35.80885844599574, -78.78616656055897), (35.810486590860904, -78.78771703319612), (35.81084461508845, -78.78586375986981), (35.8196252, -78.8083541), (35.820148, -78.798696), (35.761411, -78.782208), (35.8258283, -78.7979763), (35.78126, -78.757962), (35.781365, -78.757814), (35.782027, -78.757356), (35.780815, -78.7575), (35.80957, -78.840805), (35.78106, -78.757346), (35.780946, -78.757238), (35.826494, -78.794894), (35.822051, -78.828216), (35.828631, -78.799967), (35.819353, -78.769492), (35.830019, -78.799133), (35.815298, -78.842098), (35.793985, -78.750329), (35.830871, -78.791809), (35

In [ ]:
import math

# chat gpt gnerated function:
def haversine(lat1, lon1, lat2, lon2):
    # convert degrees → radians
    φ1, λ1, φ2, λ2 = map(math.radians, (lat1, lon1, lat2, lon2))
    dφ = φ2 - φ1
    dλ = λ2 - λ1
    a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    return 2 * 6371 * math.asin(math.sqrt(a))  # R≈6371 km

# chat gpt gnerated function:
def match_csv_api(df: pd.DataFrame,
                  api_points: list[tuple[float,float]],
                  tol_km: float = 0.1
                 ):
    records = []
    for lat, lon in zip(df['Latitude'], df['Longitude']):
        # compute all distances
        dists = [haversine(lat, lon, plat, plon) for plat, plon in api_points]
        min_dist = min(dists)
        if min_dist <= tol_km:
            idx = dists.index(min_dist)
            matched, api_lat, api_lon = True, api_points[idx][0], api_points[idx][1]
        else:
            matched, api_lat, api_lon = False, None, None

        records.append({
            'matched':    matched,
            'api_lat':    api_lat,
            'api_lon':    api_lon,
            'dist_km':    min_dist
        })

    return pd.concat([df.reset_index(drop=True),
                      pd.DataFrame(records)],
                     axis=1)

tol_km = 0.1
matched_df = match_csv_api(df, points, tol_km)

In [ ]:
matched_only = matched_df[matched_df['matched']]

# Rename to match your matched_df keys
api_df = api_df.rename(columns={
    "Latitude":  "api_lat",
    "Longitude": "api_lon"
})

# Drop exact-duplicate lat/lon rows
api_df_unique = api_df.drop_duplicates(subset=["api_lat","api_lon"])

# Merge
combined = (
    matched_only
      .merge(
         api_df_unique,
         on=["api_lat","api_lon"],
         how="left",
         suffixes=("","_api"),
         validate="many_to_one"
      )
)

In [ ]:
url = ("https://api.weather.gov/gridpoints/RAH/69,57/forecast/hourly")
response = requests.get(url)
response.raise_for_status()
data = response.json()
periods = data['properties']['periods']

# chat gpt gnerated:
forecast = pd.json_normalize(periods)

forecast = pd.json_normalize(
    periods,
    sep = '_',
    errors = 'ignore'
)

# inspect
forecast.tail()

,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,windDirection,icon,shortForecast,detailedForecast,probabilityOfPrecipitation_unitCode,probabilityOfPrecipitation_value,dewpoint_unitCode,dewpoint_value,relativeHumidity_unitCode,relativeHumidity_value
151,152,,2025-05-10T12:00:00-04:00,2025-05-10T13:00:00-04:00,True,68,F,,7 mph,NE,https://api.weather.gov/icons/land/day/rain_sh...,Chance Rain Showers,,wmoUnit:percent,37,wmoUnit:degC,8.888889,wmoUnit:percent,49
152,153,,2025-05-10T13:00:00-04:00,2025-05-10T14:00:00-04:00,True,69,F,,7 mph,NE,https://api.weather.gov/icons/land/day/rain_sh...,Chance Rain Showers,,wmoUnit:percent,37,wmoUnit:degC,8.888889,wmoUnit:percent,47
153,154,,2025-05-10T14:00:00-04:00,2025-05-10T15:00:00-04:00,True,71,F,,8 mph,NE,https://api.weather.gov/icons/land/day/rain_sh...,Chance Rain Showers,,wmoUnit:percent,41,wmoUnit:degC,8.888889,wmoUnit:percent,44
154,155,,2025-05-10T15:00:00-04:00,2025-05-10T16:00:00-04:00,True,72,F,,8 mph,E,https://api.weather.gov/icons/land/day/rain_sh...,Chance Rain Showers,,wmoUnit:percent,41,wmoUnit:degC,8.888889,wmoUnit:percent,42
155,156,,2025-05-10T16:00:00-04:00,2025-05-10T17:00:00-04:00,True,72,F,,8 mph,E,https://api.weather.gov/icons/land/day/rain_sh...,Chance Rain Showers,,wmoUnit:percent,41,wmoUnit:degC,8.888889,wmoUnit:percent,42


In [ ]:
!pip install matplotlib

In [ ]:
!pip install meteostat pandas

In [ ]:
from datetime import datetime
from meteostat import Point, Daily
from meteostat import Hourly

rdu = Point(35.7968864, -78.8080444, 125)

start = datetime(2012, 1, 1)
end   = datetime(2022, 12, 31)

wx  = Hourly(rdu, start, end).fetch()
wx.head()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2012-01-01 00:00:00,10.6,2.2,56.0,NaN,NaN,NaN,0.0,NaN,1019.1,NaN,NaN
2012-01-01 01:00:00,8.9,2.2,63.0,0.0,NaN,NaN,0.0,NaN,1019.8,NaN,NaN
2012-01-01 02:00:00,7.8,2.3,68.0,0.0,NaN,100.0,5.4,NaN,1020.4,NaN,NaN
2012-01-01 03:00:00,6.7,2.2,73.0,0.0,NaN,NaN,0.0,NaN,1020.9,NaN,NaN
2012-01-01 04:00:00,6.1,1.6,73.0,0.0,NaN,NaN,0.0,NaN,1021.0,NaN,NaN


In [ ]:
combined.rename(columns={"Start Date/Time": "StartDateTime"}, inplace=True)
combined.head()

,StartDateTime,Station Name,Charging Time,Energy (kWh),Address 1,Address 2,City,State/Province,Zip/Postal Code,Fee,...,dist_km,StationAPI_ID,Power_kW,Amps,Volate,NumConnectors,CurrentTypeID,ConnectionTypeID,UsageTypeID,Model Number_api
0,2021-04-06T11:20:20-04:00,TOWN OF CARY / BOND PARK,03:30:52,11.211,801 High House Road,Bond Park Boathouse,Cary,North Carolina,27513,0,...,0.000212,123499,3.7,16.0,230.0,2.0,10.0,1.0,1.0,ChargePoint
1,2021-04-05T19:21:09-04:00,TOWN OF CARY / P2_DTCARYDECKE1,00:22:20,2.279,113 Walnut St,NaN,Cary,North Carolina,27511,0,...,0.025397,242301,3.7,16.0,230.0,2.0,10.0,1.0,1.0,ChargePoint
2,2021-04-04T12:10:59-04:00,TOWN OF CARY / P2_DTCARYDEKCE2,05:42:36,29.225,113 Walnut St,NaN,Cary,North Carolina,27511,0,...,0.025496,242301,3.7,16.0,230.0,2.0,10.0,1.0,1.0,ChargePoint
3,2021-04-04T16:19:17-04:00,TOWN OF CARY / BOND PARK,00:54:55,6.633,801 High House Road,Bond Park Boathouse,Cary,North Carolina,27513,0,...,0.000212,123499,3.7,16.0,230.0,2.0,10.0,1.0,1.0,ChargePoint
4,2021-04-04T07:30:32-04:00,TOWN OF CARY / TOWNHALLEAST,00:36:17,3.709,228 Ambassador Loop,NaN,Cary,North Carolina,27513,0,...,0.000361,222251,3.7,16.0,230.0,2.0,10.0,1.0,1.0,ChargePoint


In [ ]:
combined['StartDateTime'] = pd.to_datetime(combined['StartDateTime'], utc=True)

combined['StartDateTime_NY'] = combined['StartDateTime'].dt.tz_convert('America/New_York')

combined['hour_ts'] = combined['StartDateTime_NY'].apply(
    lambda ts: ts.replace(minute=0, second=0, microsecond=0)
)
wx = wx.tz_localize('UTC').tz_convert('America/New_York').reset_index()
combined = combined.merge(
    wx[['time','temp','prcp','wspd','rhum']],
    left_on='hour_ts', right_on='time', how='left'
)
combined = add_time_features(combined, DATETIME_COL)

In [ ]:
!pip install holidays

In [ ]:
import holidays
us_holidays = holidays.US()

nc_holidays = holidays.US(state='NC')

combined['date'] = combined['StartDateTime'].dt.date

combined['is_holiday'] = combined['date'].isin(nc_holidays)

combined['holiday_name'] = combined['date'].map(nc_holidays).fillna('None')
combined.head()

,StartDateTime,Station Name,Charging Time,Energy (kWh),Address 1,Address 2,City,State/Province,Zip/Postal Code,Fee,...,month,is_weekend,hour_sin,hour_cos,month_sin,month_cos,Duration_min,date,is_holiday,holiday_name
0,2021-04-06 15:20:20+00:00,TOWN OF CARY / BOND PARK,03:30:52,11.211,801 High House Road,Bond Park Boathouse,Cary,North Carolina,27513,0,...,4,0,2.588190e-01,-0.965926,0.866025,-0.5,210.866667,2021-04-06,False,None
1,2021-04-05 23:21:09+00:00,TOWN OF CARY / P2_DTCARYDECKE1,00:22:20,2.279,113 Walnut St,NaN,Cary,North Carolina,27511,0,...,4,0,-9.659258e-01,0.258819,0.866025,-0.5,22.333333,2021-04-05,False,None
2,2021-04-04 16:10:59+00:00,TOWN OF CARY / P2_DTCARYDEKCE2,05:42:36,29.225,113 Walnut St,NaN,Cary,North Carolina,27511,0,...,4,1,1.224647e-16,-1.000000,0.866025,-0.5,342.600000,2021-04-04,False,None
3,2021-04-04 20:19:17+00:00,TOWN OF CARY / BOND PARK,00:54:55,6.633,801 High House Road,Bond Park Boathouse,Cary,North Carolina,27513,0,...,4,1,-8.660254e-01,-0.500000,0.866025,-0.5,54.916667,2021-04-04,False,None
4,2021-04-04 11:30:32+00:00,TOWN OF CARY / TOWNHALLEAST,00:36:17,3.709,228 Ambassador Loop,NaN,Cary,North Carolina,27513,0,...,4,1,9.659258e-01,-0.258819,0.866025,-0.5,36.283333,2021-04-04,False,None


In [ ]:
print(combined.columns)

Index(['StartDateTime', 'Station Name', 'Charging Time', 'Energy (kWh)',
       'Address 1', 'Address 2', 'City', 'State/Province', 'Zip/Postal Code',
       'Fee', 'Model Number', 'Latitude', 'Longitude', 'geopoint', 'StationID',
       'matched', 'api_lat', 'api_lon', 'dist_km', 'StationAPI_ID', 'Power_kW',
       'Amps', 'Volate', 'NumConnectors', 'CurrentTypeID', 'ConnectionTypeID',
       'UsageTypeID', 'Model Number_api', 'StartDateTime_NY', 'hour_ts',
       'time', 'temp', 'prcp', 'wspd', 'rhum', 'hour', 'dow', 'month',
       'is_weekend', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
       'Duration_min', 'date', 'is_holiday', 'holiday_name'],
      dtype='object')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import joblib

# ─────────────────── 1. Load or grab your DataFrame ──────────────
df = combined.copy()

In [ ]:
# ─────────────────── 2. Target & feature lists ───────────────────
TARGET = "Energy (kWh)"

numeric_cols = [
    "Latitude", "Longitude", "dist_km",
    "Power_kW", "Amps", "Volate", "NumConnectors",
    "temp", "prcp", "wspd", "rhum",
    "hour", "dow", "month", "hour_sin", "hour_cos",
    "month_sin", "month_cos", "Duration_min",
    "is_weekend", "is_holiday",
    "Fee"
]

categorical_cols = [
    "StationID",
    "Model Number",
    "Model Number_api",
    "CurrentTypeID", "ConnectionTypeID", "UsageTypeID",
    "holiday_name"
]

df = df.dropna(subset=["Latitude", "Longitude", "Duration_min", TARGET])

In [ ]:
# ─────────────────── 3. Prepare X and y ───────────────────────────
X = df[numeric_cols + categorical_cols]
y = df[TARGET]

# ─────────────────── 4. Train-test split (random 80/20) ──────────
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

In [ ]:
# ─────────────────── 5. Preprocessing + model pipeline ───────────
preproc = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ]
)

reg = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=500,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist"
)

pipe = Pipeline([
    ("pre",   preproc),
    ("model", reg),
])

# ─────────────────── 6. Fit ───────────────────────────────────────
pipe.fit(X_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['Latitude', 'Longitude',
                                                   'dist_km', 'Power_kW',
                                                   'Amps', 'Volate',
                                                   'NumConnectors', 'temp',
                                                   'prcp', 'wspd', 'rhum',
                                                   'hour', 'dow', 'month',
                                                   'hour_sin', 'hour_cos',
                                                   'month_sin', 'month_cos',
                                                   'Duration_min', 'is_weekend',
                                                   'is_holiday', 'Fee']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Stati...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.03,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=500, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [ ]:
# ─────────────────── 7. Evaluate ──────────────────────────────────
def metrics(label, X_, y_):
    p = pipe.predict(X_)
    print(f"{label:<5} MAE={mean_absolute_error(y_, p):6.3f} kWh   "
          f"RMSE={mean_squared_error(y_, p):6.3f} kWh   "
          f"R²={r2_score(y_, p):6.3f}")

print("\nPerformance:")
metrics("Train", X_train, y_train)
metrics("Test ", X_test,  y_test)


Performance:
Train MAE= 1.419 kWh   RMSE= 4.105 kWh   R²= 0.961
Test  MAE= 1.713 kWh   RMSE= 7.297 kWh   R²= 0.923


In [ ]:
# ─────────────────── 8. Save pipeline ─────────────────────────────
joblib.dump(pipe, "ev_energy_prediction.joblib")
print("\nSaved model → ev_energy_prediction_pipeline.joblib")


Saved model → ev_energy_prediction_pipeline.joblib


In [ ]:
import joblib, pandas as pd

pipe = joblib.load(path + "ev_energy_prediction.joblib")

def predict_energy(feature_row: dict) -> float:
    """feature_row must supply all numeric+categorical columns used above"""
    X_new = pd.DataFrame([feature_row])
    return pipe.predict(X_new)[0]

# example:
est_kwh = predict_energy({
    "Latitude": 35.79,
    "Longitude": -78.81,
    "dist_km": 0.04,
    "Power_kW": 3.7,
    "Amps": 16,
    "Volate": 230,
    "NumConnectors": 1,
    "temp": 27.0,
    "prcp": 0.0,
    "wspd": 3.0,
    "rhum": 60.0,
    "hour": 14,
    "dow": 2,
    "month": 5,
    "hour_sin": 0.5,
    "hour_cos": -0.866,
    "month_sin": 0.5,
    "month_cos": 0.866,
    "Duration_min": 45,
    "is_weekend": 0,
    "is_holiday": 0,
    "Fee": 0,
    "StationID": 123,
    "Model Number": "CT4020-HD-GW",
    "Model Number_api": "CT4020-HD-GW",
    "CurrentTypeID": 10,
    "ConnectionTypeID": 1,
    "UsageTypeID": 1,
    "holiday_name": "None"
})
print(f"Predicted energy: {est_kwh:.2f} kWh")

Predicted energy: 4.21 kWh


#**TIme Prediciton**


In [ ]:

df = df.dropna(subset=["Duration_min", "Energy (kWh)"])

df["DesiredEnergy"] = df["Energy (kWh)"]
TARGET = "Duration_min"

numeric_cols = [
    "Latitude", "Longitude", "dist_km",
    "Power_kW", "Amps", "Volate", "NumConnectors",
    "temp", "prcp", "wspd", "rhum",
    "hour", "dow", "month", "hour_sin", "hour_cos",
    "month_sin", "month_cos",
    "is_weekend", "is_holiday", "Fee",
    "DesiredEnergy"
]

categorical_cols = [
    "StationID",
    "Model Number", "Model Number_api",
    "CurrentTypeID", "ConnectionTypeID", "UsageTypeID",
    "holiday_name"
]

# Feature / target matrices
X = df[numeric_cols + categorical_cols]
y = df[TARGET]

# ── 3. Train-test split ───────────────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42
)

In [ ]:
# ── 4. Build preprocessing + regressor pipeline ───────────────────────────
preprocess = ColumnTransformer(
    [("num", "passthrough", numeric_cols),
     ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)]
)

reg = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=400,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.9,
    random_state=42,
    tree_method="hist"
)

pipe = Pipeline([
    ("pre", preprocess),
    ("model", reg)
])

# ── 5. Fit model ───────────────────────────────────────────────────────────
pipe.fit(X_train, y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['Latitude', 'Longitude',
                                                   'dist_km', 'Power_kW',
                                                   'Amps', 'Volate',
                                                   'NumConnectors', 'temp',
                                                   'prcp', 'wspd', 'rhum',
                                                   'hour', 'dow', 'month',
                                                   'hour_sin', 'hour_cos',
                                                   'month_sin', 'month_cos',
                                                   'is_weekend', 'is_holiday',
                                                   'Fee', 'DesiredEnergy']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Stat...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=400, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [ ]:
# ── 6. Evaluate ────────────────────────────────────────────────────────────
def show_metrics(split, X_, y_):
    preds = pipe.predict(X_)
    print(f"{split:<6} MAE = {mean_absolute_error(y_, preds):6.2f} min",
          f"RMSE = {mean_squared_error(y_, preds):6.2f} min",
          f"R² = {r2_score(y_, preds):6.3f}", sep="   ")

print("\nPerformance:")
show_metrics("Train", X_train, y_train)
show_metrics("Test ", X_test,  y_test)


Performance:
Train  MAE =  13.80 min   RMSE = 420.31 min   R² =  0.961
Test   MAE =  17.95 min   RMSE = 808.35 min   R² =  0.916


In [ ]:
joblib.dump(pipe, "ev_duration_prediction.joblib")
print("\nSaved model → ev_duration_prediction_pipeline.joblib")


Saved model → ev_duration_prediction_pipeline.joblib
